In [3]:
# Plot colors
lblue="#a6cee3"
blue = "#1f78b4"
lgreen = "#b2df8a"
green = "#33a02c"
black= "#424242"


from os import path, remove
import sys
import subprocess
from subprocess import call,check_output, DEVNULL
from pandas import read_csv
from os import path
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MaxNLocator

def validate_args(datasetSize, distribution, parameter, searchAlgo, recordSizeBytes, nThreads):
    if recordSizeBytes not in [8,32,128]:
        print("The valid record size options are 8,32,128")
        exit()

# Adds a experiment to the tsv configuration file. Each experiment described the dataset 
# that is going to be searched and which search algorithm is going to be used.
# Input Args:
#             tsvpath: the path of the tsvfile
#             datasetSize: number of records of the dataset to be searched
#             distribution: distribution of the keys of the dataset to be searched
#             parameter:    datasets parameter
#             searchAlgo: name of the search algorithm to use
#             recordSizeBytes: size of record (key + payload)
#             nThreads: number of threads to be used for search
def add_to_tsv(tsvpath, datasetSize, distribution, parameter, searchAlgo, recordSizeBytes, nThreads):
    validate_args(datasetSize, distribution, parameter,searchAlgo, recordSizeBytes, nThreads)
    if not path.exists(tsvpath):
        with open(tsvpath, "w") as f:
            f.write("DatasetSize\tDistribution\tParameter\tSearchAlgorithm\tRecordSizeBytes\t#threads\n")
#     if parameter is "":
#         parameter = parameterMap[distribution]
#     distribution = paperToBenchmarkDistributionNames[distribution]
    with open(tsvpath, "a") as f:
        conf = str(datasetSize)+"\t"+ \
               str(distribution)+"\t"+ \
               str(parameter)+"\t"+ \
               str(searchAlgo)+"\t"+ \
               str(recordSizeBytes)+"\t"+ \
               str(nThreads)+"\n"
        f.write(conf)
    
def rm_tsv(tsvpath):
    if path.exists(tsvpath):
        remove(tsvpath)

def UaR_to_tsv(tsvpath, datasetSize, searchAlgo, recordSizeBytes, nThreads, parameter=42):
    add_to_tsv(tsvpath, datasetSize, "uniform", parameter, searchAlgo, recordSizeBytes, nThreads)
    
def fbids_to_tsv(tsvpath, searchAlgo, recordSizeBytes, nThreads):
    add_to_tsv(tsvpath, 1, "file", "src/datasets/fb/fb-289000.txt", searchAlgo, recordSizeBytes, nThreads)
    
def freq1_to_tsv(tsvpath, searchAlgo, recordSizeBytes, nThreads):
    add_to_tsv(tsvpath, 1, "file", "src/datasets/wf/wiki.txt", searchAlgo, recordSizeBytes, nThreads)
    
def freq2_to_tsv(tsvpath, searchAlgo, recordSizeBytes, nThreads):
    add_to_tsv(tsvpath, 1, "file", "src/datasets/wf/newman.txt", searchAlgo, recordSizeBytes, nThreads)

def fal_to_tsv(tsvpath, searchAlgo, recordSizeBytes, nThreads, parameter, datasetSize):
    add_to_tsv(tsvpath, datasetSize, "fal", parameter, searchAlgo, recordSizeBytes, nThreads)

def cfal_to_tsv(tsvpath, searchAlgo, recordSizeBytes, nThreads, parameter, datasetSize):
    add_to_tsv(tsvpath, datasetSize, "cfal", parameter, searchAlgo, recordSizeBytes, nThreads)

def gap_to_tsv(tsvpath, searchAlgo, recordSizeBytes, nThreads, random_seed, shape, datasetSize):
    add_to_tsv(tsvpath, datasetSize, "gap", str(random_seed)+","+str(shape), searchAlgo, recordSizeBytes, nThreads)
    

In [4]:
def run(tsvname):
    if not path.exists("../searchbench"):
        print("Please make sure searchbench is compiled. You can compile this by running make on the parent directory.")
        sys.exit()
        
    resultFile=tsvname+".out"
    if path.exists(resultFile):
        print("This tsv has been already executed and the results have been saved.")
        print("If you want to rerun the experiments please delete the file: "+tsvname+".out")
    else:
        with open(resultFile, "w") as log_file:
            subprocess.run(["python3","./getTimes.py","./replicate_experiments/"+tsvname], stdout=log_file, stderr=DEVNULL, cwd="../") 

In [5]:
import csv

def get_results(tsvname):
    times=[]
    with open(tsvname+".out", newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter='\t')
        for row in csvreader:
            if len(row)>0 and row[0][0].isdigit():
                times.append(float(row[0].split()[7]))
    return times


#### Figure 2

In [16]:
tsv = "fig2.tsv"
rm_tsv(tsv)

# TODO(faster)
size=5 #8
for algorithm in ["bs", "sip", "is", "tip"]:
    UaR_to_tsv(tsv, 10**size, algorithm, 8, 1)
    
for algorithm in ["bs", "sip", "is", "tip"]:
    fbids_to_tsv(tsv, algorithm, 8, 1) 
    
for algorithm in ["bs", "tip"]:
    fal_to_tsv(tsv, algorithm, 8, 1, 1.05, 10**size) 

for algorithm in ["bs", "tip"]:
    freq1_to_tsv(tsv, algorithm, 8, 1)

run(tsv)
results=get_results(tsv)
# print(results)

# X: dataset
# Y: Speedup with BS

uarResults=results[0:4]
fbidsResults=results[4:8]
falResults=results[8:10]
freq1Results=results[10:12]

# Calculate speedups of sip, is and tip compard to Binary Search
uarSpeedups = []
for i in range(1,len(uarResults)):
    uarSpeedups.append(uarResults[0]/uarResults[i])

fbidsSpeedups = []
for i in range(1,len(fbidsResults)):
    fbidsSpeedups.append(fbidsResults[0]/fbidsResults[i])

falSpeedups = []
for i in range(1,len(falResults)):
    falSpeedups.append(falResults[0]/falResults[i])
    
freq1Speedups = []
for i in range(1,len(freq1Results)):
    freq1Speedups.append(freq1Results[0]/freq1Results[i])

    
# Plot

# Group the speedups of each algorithm
tipSpeedups=[uarSpeedups[0], fbidsSpeedups[0], falSpeedups[0], freq1Speedups[0]]
ISSpeedups=[uarSpeedups[1], fbidsSpeedups[1],0,0]
sipSpeedups=[uarSpeedups[2], fbidsSpeedups[2],0,0]

labels=["UaR", "fb", "fal-1.05", "wf"]
pos = [0,1,2,3]
width = 0.25 

fig, ax = plt.subplots(figsize=(10,5))
plt.bar(pos, 
        tipSpeedups, 
        width, 
        alpha=0.5, 
        color=lblue, 
        label="TIP") 

plt.bar([p + width+.02 for p in pos], 
        ISSpeedups,
        width, 
        alpha=0.5, 
        color=blue, 
        label="Interpolation Search",
        hatch="\\") 

plt.bar([p + width*2+.04 for p in pos], 
        sipSpeedups,
        width, 
        alpha=0.5, 
        color=lgreen, 
        label="SIP",
        hatch="-") 

plt.plot([-1,5],[1,1],linestyle="dashed", color=black, label="BS")

ax.set_ylabel('Speedup', size=18)

ax.set_title('Figure 2', size=22)

# Set the position of the x ticks
ax.set_xticks([p + 1.5 * width for p in pos])
ax.yaxis.set_major_locator(MaxNLocator(integer=True))

# Set the labels for the x ticks
ax.set_xticklabels(labels, size=15)
ax.tick_params(axis='both', which='major', labelsize=16)

# Setting the x-axis and y-axis limits
plt.xlim(min(pos)-width, max(pos)+width*4)
# plt.ylim([0, max(df['pre_score'] + df['mid_score'] + df['post_score'])] )

# Adding the legend and showing the plot
plt.legend(['TIP', 'Interpolation Search', 'SIP'], loc='upper right')
plt.legend(prop={'size': 15})
plt.show()

print("Caption: Speedup achieved by Interpolation Search methods over Binary Search on real datasets.\n\
       The first two dataset UaR and fb_ids are uniformly distributed datasets, where as fal-1.05 and freq1\n\
      are skewed datasets. 8 Byte record are used.")

SyntaxError: EOL while scanning string literal (<ipython-input-16-5524e5a377e8>, line 106)

#### Figure 5

In [ ]:
# algos: i-hyp-64, i-opt-8, bs
# datasets: fb
# record-size: 8,32,128

tsv="fig5.tsv"
rm_tsv(tsv)

for recordSize in [8,32,128]:
    for algorithm in ["sip", "bs", "tip", "is"]:
        fbids_to_tsv(tsv, algorithm, recordSize, 1)

run(tsv)
results=get_results(tsv)
print(results)

# X: Record size
# Y: Time

#### Figure 6

In [ ]:
# algos: i-opt-8, bs
# datasets: UaR
# record-size: 8,32,128
    

tsv="fig6.tsv"
rm_tsv(tsv)

#TODO(faster)
for datasetSize in [3,4,5]:#,6,7,8,9]:
    for algorithm in ["bs", "sip"]:
        for recordSize in [8,32,128]:
            UaR_to_tsv(tsv, 10**datasetSize, algorithm, recordSize, 1)   
            
run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records
# Y: Speedup vs bs


#### Figure 7

In [ ]:
# algos: i-opt-8, bs
# datasets: UaR(10^3 - 10^9)
# record-size: 32
    
tsv="fig7.tsv"
rm_tsv(tsv)

#TODO(faster)
for datasetSize in [3,4,5]:#,6,7,8,9]:
    for algorithm in ["bs", "sip"]:
            UaR_to_tsv(tsv, 10**datasetSize, algorithm, 32, 1)  
    
run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records(log)
# Y: times

#### Figure 8

In [ ]:
tsv8="fig8_8.tsv"
tsv32="fig8_32.tsv"
tsv128="fig8_128.tsv"
rm_tsv(tsv8)
rm_tsv(tsv32)
rm_tsv(tsv128)

#TODO(add iseq)
for algorithm in ["sip", "iseq"]:
    for datasetSize in [3,4,5,6,7]:
        for gapShape in [0.7,0.9,0.99,0.99999]:
               gap_to_tsv(tsv8, algorithm, 8, 1, 47,gapShape, 10**datasetSize)
                
for algorithm in ["sip", "iseq"]:
    for datasetSize in [3,4,5,6,7]:
        for gapShape in [0.7,0.9,0.99,0.99999]:
               gap_to_tsv(tsv32, algorithm, 8, 1, 47,gapShape, 10**datasetSize)
                
for algorithm in ["sip", "iseq"]:
    for datasetSize in [3,4,5,6,7]:
        for gapShape in [0.7,0.9,0.99,0.99999]:
               gap_to_tsv(tsv128, algorithm, 8, 1, 47,gapShape, 10**datasetSize)
            
run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records
# Y: speedup sip vs iseq

#### Figure 9

In [ ]:
tsv="fig9.tsv"
rm_tsv(tsv)

#TODO(faster)
for algorithm in ["bs", "tip"]:
    for datasetSize in [3,4,5]:#,6,7,8,9]:
        for shape in [0.5,1.05,1.25,1.5]:
            fal_to_tsv(tsv, algorithm, 8, 1, shape, 10**datasetSize)

for algorithm in ["bs", "tip"]:
    for datasetSize in [3,4,5]:#,6,7,8,9]:
        for shape in [0.5,1.05,1.25,1.5]:
            cfal_to_tsv(tsv, algorithm, 8, 1, shape, 10**datasetSize)

run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records
# Y: times

#### Figure 10

In [ ]:
tsv="fig10.tsv"
rm_tsv(tsv)

#TODO(addiseq)
for algorithm in ["sip", "iseq"]:
    for datasetSize in [3,4,5]:
        for shape in [0.5,1.05,1.25,1.5]:
            fbids_to_tsv(tsv, algorithm, 8, 1)

run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records(log)
# Y: times

#### Figure 11

In [ ]:
tsv="fig11.tsv"
rm_tsv(tsv)

for algorithm in ["tip", "bs"]:
    for recordSize in [8,32,128]:
            freq1_to_tsv(tsv, algorithm, recordSize, 1)
            
for algorithm in ["tip", "bs"]:
    for recordSize in [8,32,128]:
            freq2_to_tsv(tsv, algorithm, recordSize, 1)
    

run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records(log)
# Y: times

#### Figure 12

In [ ]:
tsv="fig12.tsv"
rm_tsv(tsv)

#TODO(faster)
for datasetSize in [3,4,5]:#6,7,8]:
        UaR_to_tsv(tsv, 10**datasetSize, "sip", 8, 1,42)

run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records(log)
# Y: times

#### Figure 18

In [ ]:
tsv="fig18.tsv"
rm_tsv(tsv)

#TODO(faster)
for algorithm in ["bs", "sip"]:
    for threads in [2,8,16,32]:
        UaR_to_tsv(tsv, 10**4, algorithm, 8, threads,42)
#         UaR_to_tsv(tsv, 10**8, "sip", 8, threads,42)

run(tsv)
results=get_results(tsv)
print(results)

# X: Numver of records(log)
# Y: times